This notebook walks through how to make graphs in this repo. You can copy this example.ipynb and rename it for your own graphing work. <b> Please do each graph in a seperate notebook!</b>

1. Import the appropriate packages. We'll use plotly.graph_objects for making all of our graphs.

In [2]:
import datetime as dt
import plotly.graph_objects as go
import plotly.io as pio
import pandas as pd
import utils.utils as ut
import utils.graph_templates

from fredapi import Fred

2. We'll get data using the fredapi package. Set the path to the text file with your API key

In [3]:
GRAPH_NAME = "jolts_quits_hires"

API_KEY_PATH = ut.get_repo_root() / "fred_api_key.txt" 

fred = Fred(api_key_file = API_KEY_PATH)

3. Set the fed_2025 template as default

In [4]:
pio.templates.default = 'fed_2025'

GRAPH_NAME = "jolts_quits_hires"

GRAPH_OUTPUT_PATH = ut.get_repo_root() / "figures"

In [5]:
GRAPH_OUTPUT_PATH

WindowsPath('C:/Users/kaika/NEUFed2025/figures')

4. Use the fredapi to get the data and prepare it for graphing. Documentation on the optional parameters that can be passed to the get_series called are found here (the documentation in fredapi is out of date). 

https://fred.stlouisfed.org/docs/api/fred/series_observations.html#Description

If you get data from somewhere else thats fine too! Put the raw csv in the "raw_data" folder and read it in here. Make sure not to edit the raw data, just transform and graph it.

In [6]:
series_codes = {
    "Quits Rate": "JTSQUR",
    "Hires Rate": "JTSHIR"
}

In [7]:
today = dt.date.today()

quits = fred.get_series(
    series_id=series_codes["Quits Rate"],
    observation_start=dt.date(2000, 12, 1),
    observation_end=today
).rename("Quits Rate")

hires = fred.get_series(
    series_id=series_codes["Hires Rate"],
    observation_start=dt.date(2000, 12, 1),
    observation_end=today
).rename("Hires Rate")

joined_df = pd.concat(
    [quits, hires],
    axis=1,
    join='inner'
)

joined_df.index.name = "Date"

joined_df.tail()

,Quits Rate,Hires Rate
Date,,
2025-04-01,2.0,3.5
2025-05-01,2.0,3.4
2025-06-01,2.0,3.3
2025-07-01,2.0,3.3
2025-08-01,1.9,3.2


5. Now that all our data is ready, make the graph and have it save itself as a .html file to graph_output whenver the notebooks is rerun. The name of the file should exactly match the notebook name. For instance, this file "example.ipynb" produces the graph "example.html." Nice work, you made a graph! 

In [8]:
fig = go.Figure()

for col in joined_df.columns:
    fig.add_trace(
        go.Scatter(
            x=joined_df.index,
            y=joined_df[col],
            mode='lines',
            name=col
        )
    )

fig.update_layout(
    title = dict(text = 'JOLTS Quits and Hires Rates <br><sup>Monthly, Seasonally Adjusted</sup>'),
    xaxis_title="Date",
    yaxis_title="Rate",
)

fig.update_yaxes(
    tickformat=".1f",
    ticksuffix="%"
)

fig.update_xaxes(
    type='date',
    tickformat='%Y',
)

ut.add_end_labels(fig = fig)

fig.show()

fig.write_html(GRAPH_OUTPUT_PATH / f"{GRAPH_NAME}.html")